In [1]:
import numpy as np
import matplotlib.pylab as plt
np.set_printoptions(precision=2)

# Přednáška 4: Sekvenční odhad lineárních modelů, predikce

Uvažujme pozorovatelný náhodný proces $\{Y_t|X_t; t=1,2,\ldots\}$ s nezávislými stejně rozdělenými realizacemi $y_1, y_2,\ldots$ determinovanými známou pozorovanou veličinou $X_t$, např. regresorem. Ten může a nemusí existovat, o tom více později. Naším cílem nechť je tento proces modelovat prostřednictvím vhodného pravděpodobnostního modelu $f(y_t|x_t, \theta)$, kde $\theta$ je neznámý parametr, jehož odhad je samozřejmě pro kvalitu modelu rozhodující. Uvažujme dále, že toto modelování chceme dělat online, např. pro účely předpovědi (počasí, směnných kurzů, polohy letícího cíle aj.).

Existuje více způsobů, jak k modelování přistoupit. Například:
- počítat periodicky odhad na **datovém okně**. To může zahrnovat všechna data od počátku $t=1$ až po okamžik současný. Nevýhodou tohoto způsobu je jeho velká náročnost na paměť a postupně rostoucí výpočetní náročnost, vyžadující rychlé zpracování i extrémního množství dat v krátkém čase.
- periodicky odhadovat $\theta$ na **plovoucím datovém okně**. Jde o jeden z velmi častých přístupů, kdy sice používáme ta samá data nějakou dobu dokola, ale jejich množství je omezené. Můžeme například počítat z posledních 100 dat a předchozí data zahazovat.
- plně sekvenční odhad, kdy ke stávající informaci přidáváme pouze nejnovější data. Pokud je model snadno upočitatelný a data nemění rozdělení, je tento postup obzvlášť výhodný.

## Příklady modelů

## Bayesovské odhadování - malé repetitorium

Označme $y_t$ pozorovanou náhodnou veličinu (data, měření) v diskrétních časových okamžicích $t=0, 1,2,\ldots$, a vektor jejích hodnot $y_{0,t-1} = [y_0, y_1, \ldots, y_{t-1}]$. Budiž jednotlivé $y_t$ nezávislé a stejně rozdělené. Nechť je $y_t$ determinováno známou veličinou $x_t$ (např. regresorem) a konstatním parametrem $\theta$. Označme rovněž $x_{0,t-1} = [x_0, x_1, \ldots, y_{x-1}]$. 

Uvažujeme dále, že apriorní hustota $\pi(\theta|x_0, y_0)$ nese nějakou "vhodnou" znalost, např. z minulosti, nebo je neinformativní. Veličiny $x_0$ a $y_0$ mají roli "pseudo-dat$ (pseudopozorování, pseudoměření).

> **Věta (Bayesova)**
>
> Nechť $f(y_t|x_t, \theta)$ je hustota pravděpodobnosti $y_t|x_t,\theta$. Dále nechť $\pi(\theta|y_{0:t-1}, x_{0:t-1})$ je apriorní hustota pro $\theta$. Aposteriorní hustota pro jednokrokový update má tvar
>
$$
\begin{aligned}
\pi(\theta|y_{0:t}, x_{0,t}) 
&= 
\frac
{f(y_t|x_t, \theta) \pi(\theta|x_{0:t-1}, y_{0:t-1})}
{\int f(y_t|x_t, \theta) \pi(\theta|x_{0:t-1}, y_{0:t-1})d\theta} \\
&=
\frac
{f(y_t|x_t, \theta) \pi(\theta|x_{0:t-1}, y_{0:t-1})}
{f(y_t|x_t)} \\
&\propto
f(y_t|x_t, \theta) \pi(\theta|x_{0:t-1}, y_{0:t-1}).
\end{aligned}
$$

Poslední řádek bude náš nejoblíbenější, neboť vyjadřuje bayesovský update v jednoduchém zápisu - bez normalizující hustoty ve jmenovateli. Ta je totiž nezávislá na odhadovaném parametru $\theta$. Normalizaci si (*teoreticky*) snadno zařídíme kdykoliv.

Odvoďme si nyní bayesovský update pro více dat současně:
$$
\begin{aligned}
\pi(\theta|y_{0:1}, x_{0:1})
&\propto
f(y_1|x_1, \theta) \pi(\theta|x_{0}, y_{0}) \\
\pi(\theta|y_{0:2}, x_{0:2})
&\propto
f(y_2|x_2, \theta) \pi(\theta|x_{0:1}, y_{0:1}) \\
&\propto \pi(\theta|x_{0}, y_{0}) f(y_1|x_1, \theta) f(y_2|x_2, \theta) \\
&\vdots \\
\pi(\theta|y_{0:t}, x_{0:t})
&\propto
\pi(\theta|y_{0}, x_{0})
\prod_{\tau = 1}^{t}
f(y_{\tau}|x_{\tau}, \theta) \\
&\propto
\pi(\theta|y_{0:\tau-1}, x_{0:\tau-1})
\prod_{\tau = \widetilde{\tau}}^{t}
f(y_{\widetilde\tau}|x_{\widetilde\tau}, \theta),
\end{aligned}
$$
kde předposlední řádek říká, že můžeme pronásobit modely a pak teprve updatovat počáteční apriorno a poslední řádek říká, že totéž můžeme udělat v kterémkoliv čase. Závěr: sekvenční update po jednom datu je stejný, jako update více daty najednou.

** Úkoly **
- jak si zařídíme normalizaci v posledním řádku uvedené Bayesovy věty?
- uvědomte si, jak vypadá aposteriorno. Jaké musí mít vlastnosti. A co když vezmeme jako model nějakou distribuci, např. beta a jako apriorno např. také beta? Co můžeme o vzniklém aposteriornu říci?

## Sekvenční odhad

Výše jsme si ukázali, že teoreticky je sekvenční bayesovský update jednoduchá záležitost. Použijeme apriorní distribuci, přidáme nové pozorování, dostaneme aposteriorní a tu opět použijeme jako apriorní pro další pozorování:

$$
\pi(\theta|x_{0}, y_{0}) \xrightarrow[\text{Bayes}]{x_1, y_1}
\pi(\theta|x_{0:1}, y_{0:1}) \xrightarrow[\text{Bayes}]{x_2, y_2}
\pi(\theta|x_{0:2}, y_{0:2}) \rightarrow
\cdots \xrightarrow[\text{Bayes}]{x_t, y_t}
\pi(\theta|x_{0:t}, y_{0:t}) \rightarrow
\cdots
$$

Připomeňme dále, že jako bodový odhad $\theta$ můžeme použít například
- střední hodnotu $\mathbb{E}[\theta]$, přesněji $\mathbb{E}[\theta|x_{0:t}, y_{0:t}]$, nebo 
- modus (maximum hustoty psti) - tzv. maximum aposteriori (MAP) odhad, nebo 
- medián (prostřední hodnotu).

Jak moc tomuto odhadu věříme, neboli jak mnoho informace aposteriorní distribuce obsahuje, nám udává například rozptyl, $\text{var}\theta$.

**
Ač se to ze vzorců výše možná nezdá, zásadním problémem bayesovského odhadu (modelování) je odvození aposteriorní distribuce a jejích vlastností. A se sekvenčním odhadem se tento problém ještě umocňuje. Pokud po prvním updatu není aposteriorno některou z "běžných" distribucí, pak s dalším updatem je aposteriorno ještě komplikovanější.
**

**
Existuje ale případ, kdy výpočty půjdou velmi pohodlně. K tomu si musíme zavést exponenciální třídu distribucí a konjugovaná apriorna.
**

> **Definice (exponenciální třída distribucí)**
>
> Uvažujme náhodnou veličinu $y$ podmíněnou veličinou $x$ a parametrem $\theta$. Exponenciální třída distribucí obsahuje distribuce s hustotou pravděpodobnosti ve tvaru
> $$
f(y|x, \theta) = h(y, x) g(\theta) \exp \left[ \eta^{\intercal} T(y,x) \right],
$$
>
> kde $\eta \equiv \eta(\theta)$ je přirozený parametr, $T(y,x)$ je suficientní statistika fixního rozměru, $h(y,x)$ je známá funkce a $g(\theta)$ je normalizační funkce. Platí-li $\eta(\theta)=\theta$ je třída kanonická.


> **Definice (konjugovaná apriorní distribuce)**
>
> Nechť $y|x, \theta$ má rozdělení z exponenciální třídy distribucí. Říkáme, že apriorní distribuce $\theta$ s hyperparametry $\xi$ a $\nu$ je k němu konjugovaná, pokud její hustota pravděpodobnosti má tvar
$$
        \pi(\theta) = q(\xi, \nu) g(\theta)^{\nu} \exp \left[ \eta^{\intercal} \xi \right],
$$
>
>kde $\xi$ má stejný rozměr jako $T(y,x)$, $\nu\in\mathbb{R}^{+}$ a $q(\xi,\nu)$ je známá funkce. Funkce $g(\theta)$ je stejná jako normalizační funkce v hustotě pro $y|x, \theta$.

$$
\begin{aligned}
    \xi_{t} &= \xi_{t-1} + T(y_{t},x_{t}), \\
    \nu_{t} &= \nu_{t-1} + 1.
\end{aligned}
$$
$$
\begin{aligned}
    \xi_{t} &= \xi_{\tau-1} + \sum_{\widetilde{\tau}=\tau}^{t} T(y_{\widetilde{\tau}},x_{\widetilde{\tau}}),\\
    \nu_{t} &= \nu_{\tau-1} + t - \tau+1.
\end{aligned}
$$

## Regresní model